# Glycerol stocks from 96-well cultures

## Protocol Initialization
Ensure you have calibrated the pipettes before doing this protocol

In [ ]:
from opentrons import simulate, execute
import json

Use one of the code lines below for simulation or execution mode:

**Simulation mode**
```
protocol = simulate.get_protocol_api('2.13')
```



**Execution mode**
```
protocol = execute.get_protocol_api('2.13')
```

In [ ]:
protocol = simulate.get_protocol_api('2.13')
protocol.home()

## Declaring Labware
- **Right mount**: An 8-channel p300 pipette
- **Slot 1**: Plate with bacterial cultures (96-deepwell, 0,5mL per well)
- **Slot 4**: 1-well reservoir with 50% glycerol
- **Slot 7**: Tiprack for p300
- **Slot 2, 3, 5 and 6**: Destination plates 

Replace with appropriate labware codes in the cell below if needed
If you need to use custom labware, use our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [ ]:
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    culture_plate = protocol.load_labware_from_definition(labware_def, 1)

destination_plates = [protocol.load_labware('biorad_96_wellplate_200ul_pcr', '2'),
                      protocol.load_labware('biorad_96_wellplate_200ul_pcr', '3'),
                      protocol.load_labware('biorad_96_wellplate_200ul_pcr', '5'),
                      protocol.load_labware('biorad_96_wellplate_200ul_pcr', '6')]

# Here using 12-well reservoir for a max volume of 20mL.
# Since a 96-well run would use almost 18mL, it might be advisable to use a 1-well reservoir 
glyc_res = protocol.load_labware('nest_1_reservoir_195ml', '4')

tipracks_300 = [protocol.load_labware('opentrons_96_tiprack_300ul', '7'),
                protocol.load_labware('opentrons_96_tiprack_300ul', '8')]    

p300_8 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=tipracks_300)

# Define the number of samples here
- Run the next cell and fill out the prompts
- You can use from 1 to 12 columns.

In [ ]:
cols_no = int(input("Number of columns: "))

if not cols_no - 1 in range(12):
    cols_no = 0
    print('Invalid number of samples')

## Labware Calibration

Change the x, y, and z variables (**in mm**) in the "set_offset" functions and run each cell repeatedly until the positioning is correct for that piece of labware. You will have to use a "dummy" plate to calibrate the labware that is not on the deck yet (competent cells).

### Tipracks:

In [ ]:
protocol.comment('--- CALIBRATION ---')

In [ ]:
tipracks_300[0].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(tipracks_300[0]['A1'].top())

### Plates/Racks:

In [ ]:
p300_8.pick_up_tip()

In [ ]:
culture_plate.set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(culture_plate['A1'].top())

In [ ]:
destination_plates[0].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(destination_plates[0]['A1'].top())

In [ ]:
destination_plates[1].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(destination_plates[1]['A1'].top())

In [ ]:
destination_plates[2].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(destination_plates[2]['A1'].top())

In [ ]:
destination_plates[3].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(destination_plates[3]['A1'].top())

In [ ]:
glyc_res.set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(glyc_res['A1'].top())

In [ ]:
p300_8.return_tip()
p300_8.reset_tipracks()

## Protocol Execution
- Get glycerol, mix, and distribute to destination wells

In [ ]:
protocol.comment('--- PROTOCOL EXECUTION ---')

In [ ]:
protocol.comment('DISTRIBUTING GLYCEROL')

p300_8.pick_up_tip()

for col in range(cols_no):
    
    p300_8.aspirate(300, glyc_res['A1'])
    for plate in destination_plates:
        p300_8.dispense(75, plate.rows()[0][col])
        p300_8.touch_tip()
    p300_8.blow_out(glyc_res['A1'])
    
p300_8.drop_tip()
    
print('Done')

In [ ]:
for col in range(cols_no):
    p300_8.pick_up_tip()
    
    protocol.comment('DISTRIBUTING CELLS IN COL ' + str(col+1))
    
    p300_8.aspirate(300, culture_plate.rows()[0][col].bottom(z=3.0))
    
    for plate in destination_plates:
        p300_8.dispense(75, plate.rows()[0][col])
        
    protocol.comment('MIXING CELLS IN COL ' + str(col+1))
        
    for plate in destination_plates:
        p300_8.mix(3, 130, plate.rows()[0][col])
        
    p300_8.drop_tip()
print('Done')

In [ ]:
protocol.home()

In [ ]:
protocol.commands()